# Example unitest framework

This is a custom unitest framework allowing to test *function's consistency*. 

For models' testing, it is not possible to get a clear output you could manually encode. Therefore, classical unitest frameworks are not enough to test this kind of models. However, it could be interesting to check, after an update, that you model still works correcly !

The objective of this framework is to run a function that is *working* (does what you want) and dump its behavior on data. Later, it will load the saved output, performs again the function on the same input data and checks whether outputs match or not : if not, you have broken something :D


## Simple example

In the 1st cell, it is a simple example of comparison you can do on data types where there are 3 mistakes. As shown in the output, all tests are run and it has detected the 3 errors and tells you where they are (test name and position of the test (starting at index 0)).

In the sencond cell, I have solved the 3 errors and re-run the tests : only failed tests are re-executed and now they all succeed !

In [1]:
import numpy as np

from unitest import *

@Test(sequential = True)
def test_dict():
    assert_equal({'a' : 1, 'b' : 2, 'c' : 3}, {'a' : 1, 'b' : 2, 'c' : 3})
    assert_equal({'a' : 1, 'b' : 2, 'c' : 4}, {'a' : 1, 'b' : 2, 'c' : 3})
    assert_not_equal({'a' : 1, 'b' : 2, 'c' : 3}, {'a' : 1, 'b' : 2, 'd' : 3})

@Test
def test_primitives():
    assert_equal(1, 2)
    assert_equal(True, 1)
    assert_false(0)
    assert_true(1 == 2)
    assert_none(None)

@Test
def test_array():
    assert_equal(np.arange(5), [0, 1, 2, 3, 4])

res = run_tests(to_run = 'all', debug = True)
print(res)

test_dict run test index 0
test_dict run test index 1
test_primitives run test index 0
test_primitives run test index 1
test_primitives run test index 2
test_primitives run test index 3
test_primitives run test index 4
test_array run test index 0
Tests summary :
9 tests executed in 0.173 sec (6 passed)
Failed tests (3) :

===== test_dict =====
3 tests executed in 0.000 sec (2 passed)
Failed tests (1) :
- Test 1 failed with message Invalid items (1) :
Key c : '4' != '3'

===== test_primitives =====
5 tests executed in 0.000 sec (3 passed)
Failed tests (2) :
- Test 0 failed with message '1' != '2'
- Test 3 failed with message 'True' != 'False'



In [2]:
@Test
def test_dict():
    assert_equal({'a' : 1, 'b' : 2, 'c' : 3}, {'a' : 1, 'b' : 2, 'c' : 3})
    assert_equal({'a' : 1, 'b' : 2, 'c' : 4}, {'a' : 1, 'b' : 2, 'c' : 4})
    assert_not_equal({'a' : 1, 'b' : 2, 'c' : 3}, {'a' : 1, 'b' : 2, 'd' : 3})

@Test
def test_primitives():
    assert_equal(1, 1)
    assert_equal(True, 1)
    assert_false(0)
    assert_true(1 == 1)
    assert_none(None)

@Test
def test_array():
    assert_equal(np.arange(5), [0, 1, 2, 3, 4])

res = run_tests(debug = True)
print(res)

test_dict run test index 1
test_primitives run test index 0
test_primitives run test index 3
Tests summary :
9 tests executed in 0.173 sec (9 passed)



## More realistic tests : image processing

Following examples come from `test/test_utils_image.py` for image processing testing.

In [3]:
import os
import tensorflow as tf

from unitest import Test, assert_function, assert_equal, assert_smaller

from utils.image import load_image, augment_image
from utils.image.mask_utils import create_color_mask
from utils.image.box_utils import *

_filename = os.path.join('test', '__datas', 'lena.jpg')

_image = None

def maybe_load_image():
    global _image
    if _image is None:
        _image = load_image(_filename)
    
    return _image


### The lazy test.

The 1st test has a hardcoded target (known in advance) and the value is a function (`get_image_size`) and additional arguments (`filename`) will be given to the function when executing this test.

The 2nd one is *pure lazy test* where both target and value are calculated when executing the test. 

In [4]:
@Test
def image_io():
    image = maybe_load_image()
    
    assert_equal((512, 512), get_image_size, _filename)
    assert_equal(lambda: get_image_size(_filename), lambda: get_image_size(image))


### Random-dependant test

As we could expect, data augmentation is random (random noise, random blur, ...) but it is still possible to test it even without knowing the expected output !

The `assert_function` does not take *target* and *value* but a **function** as argument (in this case `augment_image`) with all `args` and `kwargs` passed to the function. 

When the test will be executed for the 1st time : 
1. Run the function with given parameters.
2. Save the produced output in a file

When the test will be re-executed later :
1. Reload the saved output
2. Run the function with given parameters.
3. Compare loaded output with computed output.

**Important Note** : it is necessary that your function is reproducible (with `seed`) otherwise you cannot reproduce an output ;)

In [5]:
@Test(contains_randomness = True)
def test_image_augmentation():
    image = maybe_load_image()
    
    for i, transform in enumerate(['color', 'flip_horizontal', 'flip_vertical', 'noise', 'hue', 'saturation', 'brightness', 'contrast']):
        assert_function(augment_image, image, transform, 1., seed = i)

### Sequential test

This test comes from `test/test_utils_text.py` for `TextEncoder` testing.

This test also performs *function consistency* (as in the previous example) but with a specificity : it is sequential. It means that if a test craches, it will not execute the next ones. The logic behind this feature is that if the encoder failed to re-encode correctly the 1st sentence, it will also fail for next ones, no need to test them.

In [6]:
_default_sentences  = [
    "Hello World !",
    "Bonjour à tous !",
    "1, 2, 3, 4, 5, 6, 7, 8, 9 et 10 !",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.\
    Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure \
    dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident,\
    sunt in culpa qui officia deserunt mollit anim id est laborum."
]

def test_text_encoder(encoder):
    set_sequential()
    for sent in _default_sentences:
        assert_function(encoder.encode, sent)
        assert_function(lambda text: encoder.decode(encoder.encode(text)), sent)
        assert_function(encoder.split, sent, max_length = 150)
    
    assert_function(encoder.join, * _default_sentences)

@Test
def test_english_text_encoder():
    from utils.text import default_english_encoder
    
    test_text_encoder(default_english_encoder())


In [7]:
res = run_tests()
print(res)
res.assert_succeed()

Tests summary :
34 tests executed in 0.626 sec (34 passed)

All tests succeed !


## Run all tests

In [ ]:
from test import *
from unitest import run_tests

run_tests(debug = True).assert_succeed()